In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import tensorflow as tf

In [2]:
dt=pd.read_csv("ghi.csv")

In [3]:
dt.drop('Unnamed: 0',axis=1,inplace=True)

In [4]:
dt

,ttime,pm1,pm2,pm3,am,sm,st,lum,humd,temp,pres
0,2022-07-18 15:42:55,1.931816,-2.019171,-2.115198,0.858160,7813.0,-0.147317,2.865778,0.464013,0.208479,0.094283
1,2022-07-18 15:44:55,-0.197695,-2.019171,-2.115198,-0.633184,7576.0,-0.145832,1.749176,0.418095,0.255715,0.094204
2,2022-07-18 15:47:55,2.464194,-2.019171,-2.115198,1.603832,7692.0,-0.144348,2.598377,0.353182,0.273559,0.093909
3,2022-07-18 15:49:55,0.334683,-2.019171,-2.115198,-0.633184,7576.0,-0.142863,2.310062,0.288270,0.291404,0.093615
4,2022-07-18 15:52:55,2.987066,-2.019171,-2.115198,1.230996,7813.0,-0.141379,2.580753,0.302255,0.289304,0.092781
...,...,...,...,...,...,...,...,...,...,...,...
39500,2023-03-10 09:39:09,-0.463884,-1.159500,-1.107009,0.485324,291.0,0.918589,-0.568855,-2.162945,1.479638,0.216511
39501,2023-03-10 09:55:12,0.068494,-1.159500,-1.146289,1.603832,327.0,0.908197,1.141060,-2.252695,1.641288,0.216311
39502,2023-03-10 10:06:09,-0.996262,-1.159500,-1.107009,0.112488,291.0,0.897805,-0.568855,-2.281916,1.796641,0.215181
39503,2023-03-10 10:23:12,-0.996262,-0.725792,-1.126826,1.230996,327.0,-3.432621,1.311417,-2.311136,1.951993,0.214051


In [5]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
  dataset = tf.data.Dataset.from_tensor_slices(series)
  dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
  dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
  dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1]))
  dataset = dataset.batch(batch_size).prefetch(1)
  return dataset

In [6]:
column_indices = {name: i for i, name in enumerate(dt.columns)}

n = len(dt)
train_df = dt[0:int(n*0.7)]
val_df = dt[int(n*0.7):int(n*0.9)]
test_df = dt[int(n*0.9):]


In [7]:
train_df['sm']

0        7813.0
1        7576.0
2        7692.0
3        7576.0
4        7813.0
          ...  
27648     709.0
27649     720.0
27650     707.0
27651     720.0
27652     708.0
Name: sm, Length: 27653, dtype: float64

In [8]:
series=dt['sm'].to_numpy()

In [12]:
split_time=10000
x_train = series[:split_time]

In [19]:
window_size = 20
batch_size = 24
shuffle_buffer_size = 1000
dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)
print(dataset)
l0 = tf.keras.layers.Dense(1, input_shape=[window_size])
model = tf.keras.models.Sequential([l0])


model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam())
model.fit(dataset,epochs=100,verbose=0)

print("Layer weights {}".format(l0.get_weights()))

<PrefetchDataset element_spec=(TensorSpec(shape=(None, None), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>
Layer weights [array([[-0.01679292],
       [-0.05198202],
       [ 0.08104928],
       [ 0.06898172],
       [ 0.0478209 ],
       [ 0.05565329],
       [-0.09496845],
       [-0.13576481],
       [ 0.152557  ],
       [ 0.09179574],
       [-0.11049156],
       [-0.00404802],
       [ 0.1490968 ],
       [ 0.08319507],
       [ 0.11789661],
       [-0.07785937],
       [-0.00202662],
       [ 0.07260349],
       [ 0.6228725 ],
       [-0.050607  ]], dtype=float32), array([0.2111486], dtype=float32)]


In [ ]:
model.save(new_model.h5)

In [ ]:
forecast = []

for time in range(len(series) - window_size):
  forecast.append(model.predict(series[time:time + window_size][np.newaxis]))

forecast = forecast[split_time-window_size:]
results = np.array(forecast)[:, 0, 0]


plt.figure(figsize=(10, 6))

plot_series(time_valid, x_valid)
plot_series(time_valid, results)

1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 14ms/step


...

1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 17ms/step


1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 14ms/step


In [ ]:
x_valid = series[split_time:]